# Sparse Systems and Iterative Methods

We'll look at `scipy.sparse` along with some basic principles of iterative methods, which are at the heart of many production-level simulation tools (e.g., most finite-element modelers).

## Required Preparation


- Review [linear systems](http://robertsj.github.io/python_for_engineers/courses/pythonic_apps_2/modules/module_2/module_2.html)


## Project 1

 - Teams of 2 or 3 
 - Problem must be central to a core ME class (e.g., thermo, fluids, heat transfer, machine design)
 - Solution must incorporate something from SciPy, collaborate version control, and unit tests.  *Can* include object-oriented programming and GUIs.
 - Half-page (max) proposal due by 10/11.
 - Final project with three-page (max) report due by 11/08.
 
 
Goal here is on the *development practice* and not on how challenging the problem is, how complicated the solution is, etc.  Breadth and depth will be checked at proposal stage.

# Why Sparse Systems?

Consider our model problem $-k y'' = q$ subject to $y(0) = y_L$ and $y(1) = y_R$.  Application of the central difference led to

$$
 \left (  \begin{matrix}
  1 & 0 & \cdots & &\\
 -\frac{k}{\Delta^2} & \frac{2k}{\Delta^2} & -\frac{k}{\Delta^2} & 0 & \ldots \\
 0 & -\frac{k}{\Delta^2} & \frac{2k}{\Delta^2} & -\frac{k}{\Delta^2} & 0 & \ldots \\
  &   & \ddots &   & \ddots \\
  &   &  & \ldots & 0 & 1\\
 \end{matrix} \right )
 \left (  \begin{matrix}
  y_0 \\
  y_1 \\
  y_2 \\
  \vdots \\
  y_{n-1} \\
 \end{matrix} \right ) 
= 
 \left (  \begin{matrix}
  y_L \\
  q_1 \\
  q_2 \\
  \vdots \\
  y_R \\
 \end{matrix} \right ) 
$$

Questions:

  - How many elements of the matrix are nonzero (assume $k > 0$)?
  - If we use a regular `np.array` to store this matrix, how much does a matrix-vector product cost to compute (i.e., how many floating-point operations does it require)?  
  - Approximately what fraction of those operations are wasted?
  - Could you solve this system on your machine if $n = 10000$?

Let's construct the model system (and solve it) using tools we already have:

In [11]:
import numpy as np
def make_A_and_b(n, k=0.2, Delta = 0.1, y_L=0, y_R=0, q=1):
    """Make model """
    A = np.zeros((n, n))
    b = np.ones(n)*q
    b[0] = y_L; b[-1] = y_R
    A[0, 0] = 1.0; A[-1, -1] = 1.0
    for i in range(1, n-1):
        A[i, i] = 2*k/Delta**2; A[i, i-1] = -k/Delta**2; A[i, i+1] = -k/Delta**2
    return A, b
A, b = make_A_and_b(5, Delta=1.0, y_L=1)
np.linalg.solve(A, b)

array([ 1.  ,  8.25, 10.5 ,  7.75,  0.  ])

Now, let's explore `scipy.sparse`.  Key commands:

  - `scipy.sparse.csr_matrix`
  - `scipy.sparse.linalg.spsolve`
  - `scipy.sparse.csr_matrix`
  - `scipy.sparse.csr_matrix.todense`
  - `scipy.sparse.csr_matrix.diagonal`
  - `scipy.sparse.diags`
  - `scipy.sparse.tril` and `triu`

In [2]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg 

In [3]:
# Create a sparse matrix from A
A_sparse = sp.sparse.csr_matrix(A)
A_sparse.toarray()
import matplotlib.pyplot as plt
plt.spy(A_sparse)

In [4]:
# Solve the sparse system
sp.sparse.linalg.spsolve(A_sparse, b)

array([ 1.  ,  8.25, 10.5 ,  7.75,  0.  ])

But we don't want to build a dense matrix in the first place...

In [5]:
# Look at the structure of A_sparse
A_sparse

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [6]:
# Build A_sparse again from that structure (other ways, too)
tmp = A_sparse.data, A_sparse.indices, A_sparse.indptr
A_sparse_2 = sp.sparse.csr_matrix(tmp)
A_sparse_2.toarray()

array([[ 1. ,  0. ,  0. ,  0. ,  0. ],
       [-0.2,  0.4, -0.2,  0. ,  0. ],
       [ 0. , -0.2,  0.4, -0.2,  0. ],
       [ 0. ,  0. , -0.2,  0.4, -0.2],
       [ 0. ,  0. ,  0. ,  0. ,  1. ]])

## Iterative Methods

Consider $ax = b$.  Easy to solve, but pretend you don't have division.  How about

$$
  x^{new} = (1 - a)x^{old} + b
$$

In [7]:
a = 2.1
b = 1.0 # expect x=2.0
x = 0.0
for i in range(100):
    x = (1-a)*x + b
    print("{:.4e} {:.4e}".format(x, x-b/a))

1.0000e+00 5.2381e-01
-1.0000e-01 -5.7619e-01
1.1100e+00 6.3381e-01
-2.2100e-01 -6.9719e-01
1.2431e+00 7.6691e-01
-3.6741e-01 -8.4360e-01
1.4042e+00 9.2796e-01
-5.4457e-01 -1.0208e+00
1.5990e+00 1.1228e+00
-7.5892e-01 -1.2351e+00
1.8348e+00 1.3586e+00
-1.0183e+00 -1.4945e+00
2.1201e+00 1.6439e+00
-1.3321e+00 -1.8083e+00
2.4654e+00 1.9892e+00
-1.7119e+00 -2.1881e+00
2.8831e+00 2.4069e+00
-2.1714e+00 -2.6476e+00
3.3885e+00 2.9123e+00
-2.7274e+00 -3.2036e+00
4.0001e+00 3.5239e+00
-3.4001e+00 -3.8763e+00
4.7401e+00 4.2640e+00
-4.2142e+00 -4.6903e+00
5.6356e+00 5.1594e+00
-5.1991e+00 -5.6753e+00
6.7190e+00 6.2429e+00
-6.3909e+00 -6.8671e+00
8.0300e+00 7.5539e+00
-7.8330e+00 -8.3092e+00
9.6164e+00 9.1402e+00
-9.5780e+00 -1.0054e+01
1.1536e+01 1.1060e+01
-1.1689e+01 -1.2166e+01
1.3858e+01 1.3382e+01
-1.4244e+01 -1.4720e+01
1.6669e+01 1.6192e+01
-1.7335e+01 -1.7812e+01
2.0069e+01 1.9593e+01
-2.1076e+01 -2.1552e+01
2.4183e+01 2.3707e+01
-2.5602e+01 -2.6078e+01
2.9162e+01 2.8686e+01
-3.1078e+01 

For matrices, this becomes Richardson iteration:

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{A})\mathbf{x}^{old} + \mathbf{b} \, .
$$

Works only if $||\mathbf{I}-\mathbf{A}|| < 1$, which is not true for many systems, including our model problem!

Ultimately, iteration is good when (1) matrices are *sparse* and (2) the number of iterations is small.

In [18]:
x_sol = sp.sparse.linalg.spsolve(A_sparse, b)

x = sp.zeros(5)
for i in range(100):
    x = x - A_sparse@x + b
    print("norm of error = {:.4e}".format(sp.linalg.norm(x-x_sol)))
    
print(x_sol)
print(x)

norm of error = 1.3725e+01
norm of error = 1.2113e+01
norm of error = 1.0692e+01
norm of error = 9.4393e+00
norm of error = 8.3333e+00
norm of error = 7.3569e+00
norm of error = 6.4950e+00
norm of error = 5.7340e+00
norm of error = 5.0622e+00
norm of error = 4.4692e+00
norm of error = 3.9456e+00
norm of error = 3.4833e+00
norm of error = 3.0752e+00
norm of error = 2.7149e+00
norm of error = 2.3969e+00
norm of error = 2.1160e+00
norm of error = 1.8681e+00
norm of error = 1.6493e+00
norm of error = 1.4560e+00
norm of error = 1.2855e+00
norm of error = 1.1349e+00
norm of error = 1.0019e+00
norm of error = 8.8452e-01
norm of error = 7.8089e-01
norm of error = 6.8941e-01
norm of error = 6.0864e-01
norm of error = 5.3733e-01
norm of error = 4.7438e-01
norm of error = 4.1880e-01
norm of error = 3.6974e-01
norm of error = 3.2642e-01
norm of error = 2.8818e-01
norm of error = 2.5441e-01
norm of error = 2.2461e-01
norm of error = 1.9829e-01
norm of error = 1.7506e-01
norm of error = 1.5455e-01
n

Better option: *precondition* the system with some matrix $\mathbf{P}$, i.e., solve

$$
  \mathbf{P}^{-1} \mathbf{A} \mathbf{x} = \mathbf{P}^{-1} \mathbf{b}
$$

What's the "best" possible $\mathbf{P}$?

## The Classical Schemes

Let $\mathbf{P}$ be the main diagonal $\mathbf{D}$ of $\mathbf{A}$.  Then,

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{D}^{-1}\mathbf{A})\mathbf{x}^{old} + \mathbf{D}^{-1}\mathbf{b} \, .
$$

This is **Jacobi iteration**.

In [25]:
D_inv = sp.sparse.diags(1.0/A_sparse.diagonal(0))

x_sol = sp.sparse.linalg.spsolve(A_sparse, b)
x = sp.zeros(5)
for i in range(100):
    x = x - D_inv@(A_sparse@x) + D_inv*b
    print("{} norm of error = {:.4e}".format(i, sp.linalg.norm(x-x_sol)))
    
print(x_sol)
print(x)

0 norm of error = 1.1164e+01
1 norm of error = 7.8899e+00
2 norm of error = 5.5790e+00
3 norm of error = 3.9449e+00
4 norm of error = 2.7895e+00
5 norm of error = 1.9725e+00
6 norm of error = 1.3947e+00
7 norm of error = 9.8623e-01
8 norm of error = 6.9737e-01
9 norm of error = 4.9312e-01
10 norm of error = 3.4869e-01
11 norm of error = 2.4656e-01
12 norm of error = 1.7434e-01
13 norm of error = 1.2328e-01
14 norm of error = 8.7172e-02
15 norm of error = 6.1640e-02
16 norm of error = 4.3586e-02
17 norm of error = 3.0820e-02
18 norm of error = 2.1793e-02
19 norm of error = 1.5410e-02
20 norm of error = 1.0896e-02
21 norm of error = 7.7049e-03
22 norm of error = 5.4482e-03
23 norm of error = 3.8525e-03
24 norm of error = 2.7241e-03
25 norm of error = 1.9262e-03
26 norm of error = 1.3621e-03
27 norm of error = 9.6312e-04
28 norm of error = 6.8103e-04
29 norm of error = 4.8156e-04
30 norm of error = 3.4051e-04
31 norm of error = 2.4078e-04
32 norm of error = 1.7026e-04
33 norm of error = 1

Let $\mathbf{P}$ be the main diagonal $\mathbf{D+U}$ of $\mathbf{A}$ (i.e., everything on and below the main diagonal.  Then,

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{(D+U)}^{-1}\mathbf{A})\mathbf{x}^{old} + \mathbf{(D+U)}^{-1}\mathbf{b} \, .
$$

This is **Gauss-Seidel iteration**.

## Advanced Schemes

Several schemes exist that outperform Jacobi and Gauss Seidel. Many of these are *Krylov subspace* methods, e.g., GMRES.  All seek a solution $\mathbf{x} \in \mathcal{K}_n = [\mathbf{x}_0, \mathbf{Ax}_0, \mathbf{A}^2\mathbf{x}_0, \ldots \mathbf{A}^{n-1}\mathbf{x}_0]$.  

In [26]:
sp.sparse.linalg.gmres?